In [575]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
import json
import time
import random
import string
import random
import string
from Crypto.Hash import SHA256

In [576]:
sent_messages = []


def hash_value(data):
    h = SHA256.new()
    h.update(data.encode())
    return h.hexdigest()


def xor_strings(str1, str2):
    xor_result = "".join(chr(ord(a) ^ ord(b)) for a, b in zip(str1, str2))
    return xor_result


def xor_bytes(bytes1, bytes2):
    return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])


def get_random_string(length):
    characters = string.ascii_letters + string.digits
    random_string = "".join(random.choice(characters) for _ in range(length))
    return random_string


def get_timestamp():
    return str(int(time.time()))


def send_message(message):
    json_message = json.dumps(message, ensure_ascii=False)
    print("Sending message:", json_message)
    sent_messages.append(message)

In [577]:
xor_strings("hello", "world")

'\x1f\n\x1e\x00\x0b'

In [578]:
serial_numbers = []
session_keys = {}
multicast_groups = {}
num_smart_meters = 3
gateway_key = "".join(
    random.choice(string.ascii_letters + string.digits) for _ in range(16)
)


def generate_serial_number(length=10):
    characters = string.digits
    serial_number = "".join(random.choice(characters) for _ in range(length))
    return serial_number


def generate_smart_meter(smart_meter_id, gateway_key, serial_number):

    h_sn = hash_value(serial_number)
    x_i = get_random_string(16)
    n_i = xor_strings(
        hash_value(smart_meter_id + h_sn),
        hash_value(gateway_key + x_i),
    )
    return {
        "h_sn": h_sn,
        "n_i": n_i,
        "x_i": x_i,
        "serial_number": serial_number,
    }




In [579]:
def generate_smart_meters(num_smart_meters, gateway_key):
    smart_meter_ids = []
    serial_numbers = []
    registration_info = {}

    for i in range(num_smart_meters):
        smart_meter_id = f"SM{i+1}"
        serial_number = generate_serial_number()
        serial_numbers.append(serial_number)
        smart_meter_ids.append(smart_meter_id)

        registration_info[smart_meter_id] = generate_smart_meter(
            smart_meter_id, gateway_key, serial_number
        )
    return smart_meter_ids, registration_info, serial_numbers


smart_meter_ids, registration_info, serial_numbers = generate_smart_meters(
    num_smart_meters, gateway_key
)

print("Gateway Key:")
print(f" {gateway_key}")

print("\nSmart Meter Registration Information:")
for smart_meter_id, info in registration_info.items():
    print(f"Smart Meter ID: {smart_meter_id}")
    print(f" Serial Number: {info['serial_number']}")
    print(f" H_SN: {info['h_sn']}")
    print(f" N_I: {info['n_i']}")
    print(f" X_I: {info['x_i']}")
    print()

print("Smart Meter IDs:")
for i, smart_meter_id in enumerate(smart_meter_ids):
    print(f" {i+1}. {smart_meter_id}")

Gateway Key:
 ndVXendHaoHwuMZa

Smart Meter Registration Information:
Smart Meter ID: SM1
 Serial Number: 9889497304
 H_SN: d30ac374e41c73a281b18c14c51140ec80c308acb147c78b22db2818c3a9b23d
PV \VPWQ_XWQSV U
 X_I: x3uaPMSQmB77gPXq

Smart Meter ID: SM2
 Serial Number: 5243753622
 H_SN: 3069f5d7636c91354e2221785bd0cff1c25a1a06e95735fd1a11805853a0b763
 N_I: PU^PR TW QSP	  SUR_PT TVQTQ^S
 X_I: auvBMXUl642C52Vc

Smart Meter ID: SM3
 Serial Number: 2656776307
 H_SN: a5cc9bb43e30031c098d5f47f9cf42306b7bc596e6a1d70734e4e078ae53c787
QTSV SPT UP
 X_I: QH6QZcD1bZ1jfcGK

Smart Meter IDs:
 1. SM1
 2. SM2
 3. SM3


In [580]:
def smart_meter_registration(smart_meter_id, serial_number):
    h_sn = hash_value(serial_number)
    message = {"smart_meter_id": smart_meter_id, "h_sn": h_sn}
    send_message(message)


In [581]:
def gateway_registration_processing(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    h_sn = message["h_sn"]
    x_i = get_random_string(16)
    n_i = xor_strings(hash_value(smart_meter_id + h_sn), hash_value(gateway_key + x_i))

    registration_info = {"smart_meter_id": smart_meter_id, "n_i": n_i, "x_i": x_i}

    print(f"\nRegistering: {smart_meter_id}")
    print(f" N_I: {registration_info['n_i']}")
    print(f" X_I: {registration_info['x_i']}")
    print()

    response_message = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "x_i": x_i,
        "hh_sn_i": hash_value(h_sn),
    }
    send_message(response_message)


def smart_meter_authentication(smart_meter_id, h_sn, n_i, x_i, gateway_key):
    timestamp = get_timestamp()

    did_i = xor_strings(hash_value(smart_meter_id + h_sn), hash_value(x_i + timestamp))

    k_i = get_random_string(16)  

    pk_i = hash_value(xor_strings(hash_value(gateway_key + x_i + timestamp), k_i))

    b_i = hash_value(did_i + x_i + timestamp + pk_i)
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i.hex(),
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }
    send_message(message)
    return message

In [582]:
def smart_meter_authentication_gen(smart_meter_id, registration_info, gateway_key):
    timestamp = get_timestamp()

    h_sn = registration_info[smart_meter_id]["h_sn"]
    x_i = registration_info[smart_meter_id]["x_i"]
    n_i = registration_info[smart_meter_id]["n_i"]

    
    did_i = xor_strings(
        hash_value(smart_meter_id + h_sn),
        hash_value(x_i + timestamp)
    )

    k_i = get_random_string(16)  
    print(f"gateway_key: {gateway_key}, xi: {x_i}")

    pk_i = hash_value(
        xor_strings(
            hash_value(gateway_key + x_i + timestamp),
            k_i
        )
    )

    b_i = hash_value(n_i + x_i + timestamp + pk_i)
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i,
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }
    send_message(message)
    return message

In [583]:
def gateway_authentication(message, gateway_key, registration_info):
    smart_meter_id = message["smart_meter_id"]
    did_i = message["did_i"]
    b_i = message["b_i"]
    timestamp = message["timestamp"]
    pk_i = message["pk_i"]
    x_i = registration_info[smart_meter_id]["x_i"]
    h_sn = registration_info[smart_meter_id]["h_sn"]
    n_i = registration_info[smart_meter_id]["n_i"]

    current_timestamp = get_timestamp()

    if abs(int(current_timestamp) - int(timestamp)) > 300:
        print("Timestamp verification failed.")
        return

    expected_did_i = xor_strings(
        did_i,
        hash_value(x_i + timestamp)
    )
    if expected_did_i != did_i:
        print("DID_i verification failed.")
        return
    else:
        print("DID_i verification successful.")
    expected_n_i = xor_strings(
    xor_strings(
        did_i,
        hash_value(x_i + timestamp)
    ),
    hash_value(gateway_key + x_i)
)

    if expected_n_i != n_i:
        print("N_i verification failed.")
        return
    else:
        print("N_i verification successful.")

    expected_b_i = hash_value(
        xor_strings(
            hash_value(
                xor_strings(
                    did_i,
                    hash_value(x_i + timestamp)
                )
            ),
            hash_value(gateway_key + x_i)
        ) + x_i + timestamp + pk_i
    )

    expected_b_i = hash_value(n_i + x_i + timestamp + pk_i)
    if b_i != expected_b_i:
        print("B_i verification failed.")
        return
    else :
        print("B_i verification successful.")

    k_i = xor_strings(
        hash_value(gateway_key + x_i + timestamp),
        pk_i
    )

    k_gw = get_random_string(32)  # Assuming k_gw is a 16-byte (32-character) string

    pk_gw = xor_strings(
        k_gw,
        hash_value(k_i + current_timestamp + x_i)
    )

    c_i = xor_strings(
        hash_value(did_i + current_timestamp + k_i + pk_gw),
        hash_value(gateway_key + x_i)
    )
    response_message = {
        "smart_meter_id": smart_meter_id,
        "c_i": c_i,
        "timestamp": current_timestamp,
        "pk_gw": pk_gw,
    }
    send_message(response_message)

    session_key = hash_value(k_i + k_gw)
    session_keys[smart_meter_id] = session_key
    print("Shared session key for", smart_meter_id + ":", session_key)

In [584]:
def key_refreshment(smart_meter_id, session_key, gateway_key, x_i):
    new_session_key = hash_value(
        xor_strings(session_key, hash_value(gateway_key + x_i))
    )
    print("New session key for", smart_meter_id + ":", new_session_key)
    session_keys[smart_meter_id] = new_session_key
    smart_meter_authentication_gen(
        smart_meter_id,
        registration_info,
        gateway_key,
    )

In [585]:
def establish_multicast_group(group_id, smart_meter_ids, gateway_key):
    group_key = get_random_bytes(16)

    for smart_meter_id in smart_meter_ids:
        x_a = get_random_string(16)
        session_key = session_keys[smart_meter_id]
    cipher = AES.new(bytes.fromhex(session_key), AES.MODE_EAX)
    nonce = cipher.nonce.hex()
    timestamp = get_timestamp()
    encrypted_data = cipher.encrypt(
        json.dumps(
            {
                "group_id": group_id,
                "timestamp": timestamp,
                "x_a": x_a,
                "group_key": group_key,
            }
        ).encode()
    ).hex()
    tag = cipher.digest().hex()

    message = {
        "smart_meter_id": smart_meter_id,
        "timestamp": timestamp,
        "encrypted_data": nonce + encrypted_data + tag,
    }
    send_message(message)

In [586]:
# TODO: check about time stamp fix type chect all type should be strings
def join_multicast_group(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    timestamp = message["timestamp"]
    current_timestamp = get_timestamp()

    if abs(int(current_timestamp) - int(timestamp)) > 300:
        print("Timestamp verification failed.")
        return

    encrypted_data = message["encrypted_data"]
    nonce = encrypted_data[:32]
    encrypted_data = encrypted_data[32:]
    tag = encrypted_data[-32:]
    ciphertext = encrypted_data[:-32]

    session_key = session_keys[smart_meter_id]
    cipher = AES.new(
        bytes.fromhex(session_key), AES.MODE_EAX, nonce=bytes.fromhex(nonce)
    )

    try:
        plaintext = cipher.decrypt_and_verify(
            bytes.fromhex(ciphertext), bytes.fromhex(tag)
        )
        multicast_info = json.loads(plaintext.decode())
        group_id = multicast_info["group_id"]
        x_a = multicast_info["x_a"]
        group_key = multicast_info["group_key"]

        response_message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": ciphertext,
        }
        send_message(response_message)

        multicast_groups[group_id] = {
            "group_key": group_key,
            "members": [smart_meter_id],
        }
        print("Smart meter", smart_meter_id, "joined multicast group", group_id)
    except (ValueError, KeyError) as e:
        print(
            "Error occurred during multicast group join for smart meter", smart_meter_id
        )
        print("Error message:", str(e))

In [587]:

for i in range(len(smart_meter_ids)):
    smart_meter_registration(smart_meter_ids[i], serial_numbers[i])

for i in range(len(smart_meter_ids)):
    registration_request = {
        "smart_meter_id": smart_meter_ids[i],
        "h_sn": hash_value(serial_numbers[i]),
    }
    gateway_registration_processing(registration_request, gateway_key)


for i in range(len(smart_meter_ids)):
    message = smart_meter_authentication_gen(
        smart_meter_ids[i], registration_info, gateway_key
    )
    gateway_authentication(message, gateway_key, registration_info)
    
for i in range(len(smart_meter_ids)):
    if smart_meter_ids[i] in session_keys:
        key_refreshment(
            smart_meter_ids[i],
            session_keys[smart_meter_ids[i]],
            gateway_key,
            registration_info[smart_meter_ids[i]]["x_i"],
        )
    


group_id = "GROUP1"
establish_multicast_group(group_id, ["SM1", "SM2"], gateway_key)
for smart_meter_id in ["SM1", "SM2"]:
    encrypted_data = None
    for message in sent_messages:
        if message["smart_meter_id"] == smart_meter_id:
            if "encrypted_data" in message:
                encrypted_data = message["encrypted_data"]
                break
    if encrypted_data:
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": encrypted_data,
        }
        join_multicast_group(message, gateway_key)

Sending message: {"smart_meter_id": "SM1", "h_sn": "d30ac374e41c73a281b18c14c51140ec80c308acb147c78b22db2818c3a9b23d"}
Sending message: {"smart_meter_id": "SM2", "h_sn": "3069f5d7636c91354e2221785bd0cff1c25a1a06e95735fd1a11805853a0b763"}
Sending message: {"smart_meter_id": "SM3", "h_sn": "a5cc9bb43e30031c098d5f47f9cf42306b7bc596e6a1d70734e4e078ae53c787"}

Registering: SM1
 N_I:  
]\TUU S QT  W _ 	
W
VVSUQTQX	P]^R
 X_I: O09E31n6wNAZGNB9

Sending message: {"smart_meter_id": "SM1", "n_i": "\u0007\u0003\u0000\n\u0006\u0006]\\\u0001T\u0005\u0006\u0003UU\u0000S\u0000QT\u0000\u0000\u0005W\u0006\u0003\u0000_\u0000\t\u0005\u0002\n\u000eW\u0006\n\t\u0001_\u0001U\u0004\u0006U\rVV\u0002\u0005\u000f\u0006SUQTQX\u0006\tP]^R", "x_i": "O09E31n6wNAZGNB9", "hh_sn_i": "2faa06c8abad79073b4820b20ce6622daee524246ee07001bcf0ca12b132b353"}

Registering: SM2
RSTRSSVTP VSRUTQZQ_WSUUV]R^
PY [ 
 X_I: Wx3MZE7g2nFxQ9hP

Sending message: {"smart_meter_id": "SM2", "n_i": "\u

KeyError: 'SM1'